In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [2]:
vitals = pd.read_csv('./data/vitals/vitals.csv')
vitals = vitals[vitals['abbreviation']!='tempF']
vitals.reset_index(drop=True,inplace=True)
for i in vitals.abbreviation.unique():
    globals()['chartevents_{}'.format(i)] = pd.read_parquet('./data/vitals/chartevents_%s.parquet'%i)

In [3]:
labvalues = pd.read_csv('./data/labvalues/labvalues.csv')
icustays  = pd.read_parquet(icu+'icustays.parquet')

In [16]:
for i in labvalues.abbreviation.unique():
    globals()['chartevents_{}'.format(i)] = pd.read_parquet('./data/labvalues/chartevents_%s.parquet'%i)
    globals()['labevents_{}'.format(i)] = pd.read_parquet('./data/labvalues/labevents_%s.parquet'%i)

In [57]:
outputevents_uo = pd.read_parquet('./data/others/outputevents_uo.parquet')

# Lab values

In [162]:
def resample(chartevents_,labevents_,icustays):
    rt = []
    tmp_itemid = chartevents_.itemid[0]

    icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
    icustays_intime['intime'] = pd.to_datetime(icustays_intime['intime'])
    icustays_intime = icustays_intime.rename(columns={'intime':'charttime'})

    icustays_outtime = icustays[['subject_id','hadm_id','stay_id','outtime']]
    icustays_outtime['outtime'] = pd.to_datetime(icustays_outtime['outtime'])
    icustays_outtime = icustays_outtime.rename(columns={'outtime':'charttime'})

    labevents_['charttime'] = pd.to_datetime(labevents_['charttime'])

    for i in tqdm(icustays.hadm_id.unique()):
        tmp_hosp = labevents_[labevents_['hadm_id']==i]
        tmp_hosp = tmp_hosp[['subject_id','hadm_id','charttime','itemid','valuenum']]
        tmp_hosp.sort_values('charttime',ascending=True, inplace=True)

        for i in icustays[icustays['hadm_id']==i].stay_id.unique():
            tmp = chartevents_[chartevents_['stay_id']==i]
            tmp_id = icustays[icustays['stay_id']==i][['subject_id','hadm_id','stay_id']].iloc[0].to_list()
            tmp_intime = icustays_intime[icustays_intime['stay_id']==i]
            tmp_outtime = icustays_outtime[icustays_outtime['stay_id']==i]

            tmp = pd.concat([tmp, tmp_intime, tmp_outtime])

            tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values)&(tmp['charttime'].values <= tmp_outtime.charttime.values)]
            tmp.index = pd.DatetimeIndex(tmp['charttime'])

            tmp = pd.DataFrame(tmp['valuenum'].resample(rule='H', origin='start').last())
            tmp.reset_index(drop=False,inplace=True)

            if tmp.iloc[0].isna().any():
                tmp_hosp = tmp_hosp[(tmp_hosp['charttime'].values < tmp_intime.charttime.values)]
                if not tmp_hosp.empty:
                    tmp.iloc[0]['valuenum'] = tmp_hosp.iloc[-1].valuenum
            tmp[['subject_id','hadm_id','stay_id']] = tmp_id
            tmp['itemid'] = tmp_itemid
            rt.append(tmp)
    return pd.concat(rt)

In [163]:
for i,idx in enumerate(tqdm(labvalues.abbreviation.unique())):
    if not os.path.isfile('./data/resample/resample_%s.parquet'%idx):
        print("[%i/%i] Resampling %s..."%(i+1,len(labvalues.abbreviation.unique()),idx))
        globals()['resample_{}'.format(idx)] = resample(globals()['chartevents_{}'.format(idx)],globals()['labevents_{}'.format(idx)],icustays)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/resample_%s.parquet'%idx)

  0%|          | 0/28 [00:00<?, ?it/s]

[1/28] Resampling Alb...


  0%|          | 0/66239 [00:00<?, ?it/s]

[2/28] Resampling Alk_Phos...


  0%|          | 0/66239 [00:00<?, ?it/s]

...

  0%|          | 0/66239 [00:00<?, ?it/s]

[27/28] Resampling baseexcess...


  0%|          | 0/66239 [00:00<?, ?it/s]

[28/28] Resampling lactate...


  0%|          | 0/66239 [00:00<?, ?it/s]

# Vitals

In [148]:
def resample_vitals(chartevents_,icustays,itemid):
    rt = []
    icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
    icustays_intime['intime'] = pd.to_datetime(icustays_intime['intime'])
    icustays_intime = icustays_intime.rename(columns={'intime':'charttime'})

    icustays_outtime = icustays[['subject_id','hadm_id','stay_id','outtime']]
    icustays_outtime['outtime'] = pd.to_datetime(icustays_outtime['outtime'])
    icustays_outtime = icustays_outtime.rename(columns={'outtime':'charttime'})
    for i in tqdm(icustays.stay_id.unique()):
        tmp = chartevents_[chartevents_['stay_id']==i]
        tmp_id = icustays[icustays['stay_id']==i][['subject_id','hadm_id','stay_id']].iloc[0].to_list()
        tmp_intime = icustays_intime[icustays_intime['stay_id']==i]
        tmp_outtime = icustays_outtime[icustays_outtime['stay_id']==i]

        tmp = pd.concat([tmp, tmp_intime, tmp_outtime])
        tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values)&(tmp['charttime'].values <= tmp_outtime.charttime.values)]
        tmp.index = pd.DatetimeIndex(tmp['charttime'])
        tmp = pd.DataFrame(tmp['valuenum'].resample(rule='H', origin='start').last())
        tmp.reset_index(drop=False,inplace=True)
        tmp[['subject_id','hadm_id','stay_id']] = tmp_id
        tmp['itemid'] = itemid
        rt.append(tmp)
    return pd.concat(rt)

In [54]:
vitals.abbreviation.unique()

array(['PaO2', 'PaCO2', 'FiO2', 'SaO2', 'RR', 'ABPs', 'ABPd', 'NBPs',
       'NBPd', 'tempC', 'HR', 'CVP'], dtype=object)

In [56]:
for i,idx in enumerate(tqdm(vitals.abbreviation.unique())):
    if not os.path.isfile('./data/resample/resample_%s.parquet'%idx):
        print("[%i/%i] Resampling %s..."%(i+1,len(vitals.abbreviation.unique()),idx))
        globals()['resample_{}'.format(idx)] = resample_vitals(globals()['chartevents_{}'.format(idx)],icustays,idx)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/resample_%s.parquet'%idx)

  0%|          | 0/12 [00:00<?, ?it/s]

[39553979/12] Resampling FiO2...


  0%|          | 0/73181 [00:00<?, ?it/s]

[39553979/12] Resampling SaO2...


  0%|          | 0/73181 [00:00<?, ?it/s]

...

  0%|          | 0/73181 [00:00<?, ?it/s]

[39553979/12] Resampling HR...


  0%|          | 0/73181 [00:00<?, ?it/s]

[39553979/12] Resampling CVP...


  0%|          | 0/73181 [00:00<?, ?it/s]

In [64]:
resample_ABPs

,charttime,valuenum,subject_id,hadm_id,stay_id,itemid
0,2180-07-23 14:00:00,NaN,10000032,29079034,39553978,ABPs
1,2180-07-23 15:00:00,NaN,10000032,29079034,39553978,ABPs
2,2180-07-23 16:00:00,NaN,10000032,29079034,39553978,ABPs
3,2180-07-23 17:00:00,NaN,10000032,29079034,39553978,ABPs
4,2180-07-23 18:00:00,NaN,10000032,29079034,39553978,ABPs
...,...,...,...,...,...,...
42,2145-11-04 16:59:00,NaN,19999987,23865745,36195440,ABPs
43,2145-11-04 17:59:00,NaN,19999987,23865745,36195440,ABPs
44,2145-11-04 18:59:00,NaN,19999987,23865745,36195440,ABPs
45,2145-11-04 19:59:00,NaN,19999987,23865745,36195440,ABPs


In [62]:
resample_NBPs

,charttime,valuenum,subject_id,hadm_id,stay_id,itemid
0,2180-07-23 14:00:00,95.0,10000032,29079034,39553978,NBPs
1,2180-07-23 15:00:00,88.0,10000032,29079034,39553978,NBPs
2,2180-07-23 16:00:00,91.0,10000032,29079034,39553978,NBPs
3,2180-07-23 17:00:00,95.0,10000032,29079034,39553978,NBPs
4,2180-07-23 18:00:00,86.0,10000032,29079034,39553978,NBPs
...,...,...,...,...,...,...
42,2145-11-04 16:59:00,NaN,19999987,23865745,36195440,NBPs
43,2145-11-04 17:59:00,NaN,19999987,23865745,36195440,NBPs
44,2145-11-04 18:59:00,163.0,19999987,23865745,36195440,NBPs
45,2145-11-04 19:59:00,NaN,19999987,23865745,36195440,NBPs


In [83]:
# NaN 값이 있는 행을 찾고, 해당 stay_id와 charttime으로 resample_NBPs에서 값을 찾아 대체하는 함수 (merge와 numpy.where 사용)
def replace_nan_values_efficient(resample_ABPs, resample_NBPs):
    # stay_id와 charttime을 기준으로 두 DataFrame을 병합
    merged_df = resample_ABPs.merge(resample_NBPs, on=['subject_id','hadm_id','stay_id', 'charttime'], how='left', suffixes=('', '_nbp'))

    # valuenum 열에서 NaN을 대체
    merged_df['valuenum'] = np.where(pd.isna(merged_df['valuenum']), merged_df['valuenum_nbp'], merged_df['valuenum'])

    # 원래의 열만 남기고 반환
    return merged_df[['subject_id','hadm_id','stay_id', 'charttime', 'valuenum']]

# 함수를 사용하여 NaN 값을 대체
resample_SBP = replace_nan_values_efficient(resample_ABPs, resample_NBPs)
resample_DBP = replace_nan_values_efficient(resample_ABPd, resample_NBPd)

In [142]:
def resample_ffill(resample_):
    resample_['valuenum'] = resample_.groupby('stay_id')['valuenum'].ffill()
    return resample_

def resample_bfill(resample_):
    resample_['valuenum'] = resample_.groupby('stay_id')['valuenum'].bfill()
    return resample_

In [84]:
resample_SBP['valuenum'] = resample_SBP.groupby('stay_id')['valuenum'].ffill()
resample_DBP['valuenum'] = resample_DBP.groupby('stay_id')['valuenum'].ffill()

# SOFA

## GCS

In [92]:
chartevents_GCS_eye = pd.read_parquet('./data/others/chartevents_GCS_eye.parquet')
chartevents_GCS_verval= pd.read_parquet('./data/others/chartevents_GCS_verval.parquet')
chartevents_GCS_motor = pd.read_parquet('./data/others/chartevents_GCS_motor.parquet')

In [93]:
GCS = ['GCS_eye','GCS_verval','GCS_motor']
for i,idx in enumerate(tqdm(GCS)):
    if not os.path.isfile('./data/resample/resample_%s.parquet'%idx):
        print("[%i/%i] Resampling %s..."%(i+1,len(vitals.abbreviation.unique()),idx))
        globals()['resample_{}'.format(idx)] = resample_vitals(globals()['chartevents_{}'.format(idx)],icustays,idx)
        globals()['resample_{}'.format(idx)].to_parquet('./data/resample/resample_%s.parquet'%idx)

  0%|          | 0/3 [00:00<?, ?it/s]

[39553979/12] Resampling GCS_eye...


  0%|          | 0/73181 [00:00<?, ?it/s]

[39553979/12] Resampling GCS_verval...


  0%|          | 0/73181 [00:00<?, ?it/s]

[39553979/12] Resampling GCS_motor...


  0%|          | 0/73181 [00:00<?, ?it/s]

In [115]:
resample_GCS_eye = resample_ffill(resample_GCS_eye)
resample_GCS_verval = resample_ffill(resample_GCS_verval)
resample_GCS_motor = resample_ffill(resample_GCS_motor)

In [126]:
resample_GCS = pd.merge(resample_GCS_eye,resample_GCS_verval,on=['subject_id','hadm_id','stay_id','charttime'],how='left')

In [127]:
resample_GCS = pd.merge(resample_GCS,resample_GCS_motor,on=['subject_id','hadm_id','stay_id','charttime'],how='left')

In [129]:
resample_GCS['value'] = resample_GCS['valuenum_x'] + resample_GCS['valuenum_y'] + resample_GCS['valuenum']

In [131]:
resample_GCS = resample_GCS[['subject_id','hadm_id','stay_id','charttime','value']]

In [132]:
resample_GCS

,subject_id,hadm_id,stay_id,charttime,value
0,10000032,29079034,39553978,2180-07-23 14:00:00,14.0
1,10000032,29079034,39553978,2180-07-23 15:00:00,14.0
2,10000032,29079034,39553978,2180-07-23 16:00:00,14.0
3,10000032,29079034,39553978,2180-07-23 17:00:00,14.0
4,10000032,29079034,39553978,2180-07-23 18:00:00,15.0
...,...,...,...,...,...
6099429,19999987,23865745,36195440,2145-11-04 16:59:00,13.0
6099430,19999987,23865745,36195440,2145-11-04 17:59:00,13.0
6099431,19999987,23865745,36195440,2145-11-04 18:59:00,13.0
6099432,19999987,23865745,36195440,2145-11-04 19:59:00,14.0


In [143]:
resample_PaO2 = resample_ffill(resample_PaO2)
resample_PaO2 = resample_bfill(resample_PaO2)

In [145]:
resample_PaO2.isna().sum()

charttime           0
valuenum      1969899
subject_id          0
hadm_id             0
stay_id             0
itemid              0
dtype: int64

# Urine Output

In [155]:
def resample_uo(chartevents_,icustays,itemid):
    rt = []
    chartevents_['charttime'] = pd.to_datetime(chartevents_['charttime'])
    icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
    icustays_intime['intime'] = pd.to_datetime(icustays_intime['intime'])
    icustays_intime = icustays_intime.rename(columns={'intime':'charttime'})

    icustays_outtime = icustays[['subject_id','hadm_id','stay_id','outtime']]
    icustays_outtime['outtime'] = pd.to_datetime(icustays_outtime['outtime'])
    icustays_outtime = icustays_outtime.rename(columns={'outtime':'charttime'})
    for i in tqdm(icustays.stay_id.unique()):
        tmp = chartevents_[chartevents_['stay_id']==i]
        tmp_id = icustays[icustays['stay_id']==i][['subject_id','hadm_id','stay_id']].iloc[0].to_list()
        tmp_intime = icustays_intime[icustays_intime['stay_id']==i]
        tmp_outtime = icustays_outtime[icustays_outtime['stay_id']==i]

        tmp = pd.concat([tmp, tmp_intime, tmp_outtime])
        tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values)&(tmp['charttime'].values <= tmp_outtime.charttime.values)]
        tmp.index = pd.DatetimeIndex(tmp['charttime'])
        tmp = pd.DataFrame(tmp['value'].resample(rule='H', origin='start').sum())
        tmp.reset_index(drop=False,inplace=True)
        tmp[['subject_id','hadm_id','stay_id']] = tmp_id
        tmp['itemid'] = itemid
        rt.append(tmp)
    return pd.concat(rt)

In [156]:
resample_uo = resample_uo(outputevents_uo,icustays,'uo')

  0%|          | 0/73181 [00:00<?, ?it/s]

In [160]:
resample_uo

,charttime,value,subject_id,hadm_id,stay_id,itemid
0,2180-07-23 14:00:00,0.0,10000032,29079034,39553978,uo
1,2180-07-23 15:00:00,175.0,10000032,29079034,39553978,uo
2,2180-07-23 16:00:00,0.0,10000032,29079034,39553978,uo
3,2180-07-23 17:00:00,0.0,10000032,29079034,39553978,uo
4,2180-07-23 18:00:00,0.0,10000032,29079034,39553978,uo
...,...,...,...,...,...,...
42,2145-11-04 16:59:00,0.0,19999987,23865745,36195440,uo
43,2145-11-04 17:59:00,0.0,19999987,23865745,36195440,uo
44,2145-11-04 18:59:00,0.0,19999987,23865745,36195440,uo
45,2145-11-04 19:59:00,325.0,19999987,23865745,36195440,uo
